In [0]:
!pip install weighted_levenshtein

     |████████████████████████████████| 133kB 2.8MB/s 
  Created wheel for weighted-levenshtein: filename=weighted_levenshtein-0.2.1-cp36-cp36m-linux_x86_64.whl size=317509 sha256=304fd2d3731e7e9cf796f585fd85bc2e8cdd687433f6090c65a02e9732980e48
  Stored in directory: /root/.cache/pip/wheels/4c/dc/42/4e97d36cc6241cc01c5ec7869021ad487037f987102e11b6a3
Successfully built weighted-levenshtein


In [0]:
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/My Drive/ner/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from weighted_levenshtein import lev, osa, dam_lev
from string import ascii_lowercase
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np

In [0]:
# baca file train dan test
train_lines = []
test_lines = []
with open(folder+'dataTrain.txt', 'r') as f:
    train_lines = f.readlines()
with open(folder+'kalimat_POSTag.txt', 'r') as f:
    test_lines = f.readlines()

In [0]:
#baca file list/gazeteer
per_list = []
flist = open(folder+"per_list.txt", "r")
for p in flist:
    per_list.append(p[:-1])

loc_list = []
llist = open(folder+"loc_list.txt", "r")
for l in llist:
    loc_list.append(l[:-1])

org_list = []
olist = open(folder+"org_list.txt", "r")
for o in olist:
    org_list.append(o[:-1])

In [0]:
#fungsi list lookup with edit distance
alfha = 0.4
threshold = 20

insert_costs = np.full(128, 100, dtype=np.float64)
insert_costs[ord('-')] = 10
insert_costs[ord(' ')] = 10

delete_costs = np.full(128, 100, dtype=np.float64)
delete_costs[ord('-')] = 10
delete_costs[ord(' ')] = 10

substitute_costs = np.full((128,128), 50, dtype=np.float64)
for c in ascii_lowercase:
	substitute_costs[ord(c), ord(c.capitalize())] = 10
	substitute_costs[ord(c), ord(c)] = 0
	substitute_costs[ord(c.capitalize()), ord(c)] = 10
	substitute_costs[ord(c.capitalize()), ord(c.capitalize())] = 0
substitute_costs[ord('-'), ord(' ')] = 10
substitute_costs[ord(' '), ord('-')] = 10
for i in range(10):
	for j in range(10):
		if i == j:
			substitute_costs[ord(str(i)), ord(str(j))] = 0
			substitute_costs[ord(str(j)), ord(str(i))] = 0
		else:
			substitute_costs[ord(str(i)), ord(str(j))] = 10
			substitute_costs[ord(str(j)), ord(str(i))] = 10

def edit_distance_normalized_cost(word, target):
	cost = lev(word, target, insert_costs=insert_costs, delete_costs=delete_costs, substitute_costs=substitute_costs)
	return (cost + alfha) / len(target)

def check_under_threshold(cost):
	if cost <= threshold:
		return True
	else:
		return False

def check_list(sentence, pos, list):
  words = sentence.split()
  candidate = []
  candidate.append(words[pos])
  if pos >= 0:
    if pos < (len(words) - 1):
      candidate.append(words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  if (pos - 1) >= 0:
    candidate.append(words[pos - 1] + " " + words[pos])
    if pos < (len(words) - 1):
      candidate.append(words[pos - 1] + " " + words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos - 1] + " " + words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  if (pos - 2) >= 0:
    candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos])
    if pos < (len(words) - 1):
      candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  exist = False
  for l in list:
    for c in candidate:
      if check_under_threshold(edit_distance_normalized_cost(c,l)) :
        exist = True
        break
  return exist
print(check_list('Universitas Gadjah Mada',0,org_list))

True


In [0]:
# fungsi untuk konversi label NE
def convert_label(raw_ne):
    new_label = 0
    if (raw_ne=='B-PER'): new_label = 1
    if (raw_ne=='I-PER'): new_label = 2
    if (raw_ne=='B-ORG'): new_label = 3
    if (raw_ne=='I-ORG'): new_label = 4
    if (raw_ne=='B-LOC'): new_label = 5
    if (raw_ne=='I-LOC'): new_label = 6
    return new_label

In [0]:
# inisialisasi kode token dan postag
token_dict = {}
postag_dict = {}
counter_token = 0
counter_postag = 0

In [0]:
# baca data token, postag, dan label NE
train_sents = []
test_sents = []
sent = []
counter = 0
# data train
for line in train_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1], convert_label(line_part[2]))
        if line_part[0].lower() not in token_dict.keys():
            token_dict[line_part[0].lower()] = counter_token
            counter_token = counter_token+1
        if line_part[1] not in postag_dict.keys():
            postag_dict[line_part[1]] = counter_postag
            counter_postag = counter_postag+1
        #print(t)
        sent.append(t)
    else:
        #print("train sent = ")
        #print(sent)
        train_sents.append(sent)
        sent = []
        counter = counter+1

# data test
counter = 0
sent = []
for line in test_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1])
        #print(t)
        sent.append(t)
    else:
        #print("test sent = ")
        #print(sent)
        test_sents.append(sent)
        sent = []
        counter = counter+1

In [0]:
# kode untuk token/kata dan postag yang tidak muncul di data training, namun muncul di data testing
token_dict['unk'] = 9999
postag_dict['unk'] = 9999

[[('Pengamat', 'NN'), ('politik', 'NN'), ('dari', 'IN'), ('Universitas', 'NN'), ('Gadjah', 'NNP'), ('Mada', 'NNP'), (',', ','), ('Arie', 'NNP'), ('Sudjito', 'NNP'), (',', ','), ('menilai', 'VBT'), (',', ','), ('keinginan', 'NN'), ('Ketua', 'NN'), ('Umum', 'JJ'), ('Partai', 'NN'), ('Golkar', 'NNP'), ('Aburizal', 'NNP'), ('Bakrie', 'NNP'), ('untuk', 'IN'), ('maju', 'VBI'), ('kembali', 'RB'), ('sebagai', 'IN'), ('ketua', 'NN'), ('umum', 'JJ'), ('merupakan', 'VBT'), ('pemaksaan', 'NN'), ('kehendak', 'NN'), ('.', '.')], [('Menurut', 'VBT'), ('dia', 'PRP'), (',', ','), ('ada', 'VBI'), ('kesan', 'NN'), ('bahwa', 'SC'), ('Aburizal', 'NNP'), ('menggunakan', 'VBT'), ('segala', 'RB'), ('cara', 'NN'), ('untuk', 'IN'), ('memuluskan', 'VBT'), ('jalannya', 'NN'), ('kembali', 'RB'), ('menduduki', 'VBT'), ('Golkar', 'NNP'), ('1', 'CDP'), ('.', '.')], [('Hal', 'NN'), ('ini', 'DT'), (',', ','), ('kata', 'VBT'), ('Arie', 'NNP'), (',', ','), ('berpotensi', 'VBI'), ('menimbukan', 'VBT'), ('perpecahan', 'NN'

In [0]:
# fungsi untuk ekstraksi fitur dari sebuah kalimat
def word2features(sent, i):  
    word = sent[i][0]
    postag = sent[i][1]
    sentence = ''
    for w in sent:
      sentence += w[0] + " "
    if word.lower() not in token_dict.keys(): 
        word = 'unk'
    if postag not in postag_dict.keys():
        postag = 'unk'
    features = [
        #token_dict[word.lower()], # fitur kata dalam bentuk lowercase
        word.isupper(), # fitur apakah karakter pertama token merupakan huruf kapital
        word.istitle(), # fitur apakah token merupakan title
        word.isdigit(), # fitur apakah token merupakan digit
        postag_dict[postag], # fitur kode postag token
        #ord(word[0]) if len(word) > 1 else 0, #prefix 1 in ascii
        #ord(word[-1]) if len(word) > 1 else 0, #suffix 1 in ascii
        #ord(word[1]) if len(word) > 2 else 0, #prefix 2 in ascii
        #ord(word[-2]) if len(word) > 2 else 0, #suffix 2 in ascii
        #ord(word[2]) if len(word) > 3 else 0, #prefix 3 in ascii
        #ord(word[-3]) if len(word) > 3 else 0, #suffix 3 in ascii
        check_list(sentence,i - 1,per_list) if i > 0 else False,
        check_list(sentence,i - 1,loc_list) if i > 0 else False,
        check_list(sentence,i - 1,org_list) if i > 0 else False,
        check_list(sentence,i + 1,per_list) if i < len(sent) - 1 else False,
        check_list(sentence,i + 1,loc_list) if i < len(sent) - 1 else False,
        check_list(sentence,i + 1,org_list) if i < len(sent) - 1 else False,
        check_list(sentence,i,per_list), #check di list person
        check_list(sentence,i,loc_list), #check di list location
        check_list(sentence,i,org_list) #check di list organisation
    ]
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
# ekstraksi fitur data train
X_train = []
y_train = []
for s in train_sents:
    for i in range(len(s)):
        X_train.append(word2features(s,i))
        y_train.append(s[i][2])
# ekstraksi fitur data test
X_test = []
#y_test = []
for s in test_sents:
    for i in range(len(s)):
        X_test.append(word2features(s,i))
        #y_test.append(s[i][2])
#print(X_train)
#print(y_train)

In [0]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
#y_test = np.array(y_test)

# train classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Coba test, satu kata
#print('xtest')
#print(X_test[1])

#print(clf.predict(X_test[1].reshape(1,-1)))
# Coba test, keseluruhan data test
#print('hasil klasifikasi data test:')
res = clf.predict(X_test)
file = open(folder+'klasifikasi_output.txt','w') 
cc = 0
for s in test_sents:
    for w in s:
        file.write(w[0])
        #print(w[0])
        file.write('\t')
        #print(res[cc])
        if res[cc] == 0:
            file.write('O')
            #print('O')
        elif res[cc] == 1:
            file.write('B-PER')
            #print('B-PER')
        elif res[cc] == 2:
            file.write('I-PER')
            #print('B-PER')
        elif res[cc] == 3:
            file.write('B-ORG')
            #print('B-ORG')
        elif res[cc] == 4:
            file.write('I-ORG')
            #print('I-ORG')
        elif res[cc] == 5:
            file.write('B-LOC')
            #print('I-ORG')
        elif res[cc] == 6:
            file.write('I-LOC')
            #print('I-LOC')
        cc += 1
        file.write('\n')
        #print('\n')
    file.write('\n')
    #print('\n')
print('finished')
file.close()

finished
